In [ ]:
# LFS Demography Cleaning (2015–2024)
**Goal:** Turn StatCan table `14100088.csv` (Labour force characteristics by immigrant status / country of birth) 
into a tidy, annual dataset suitable for analysis alongside Job Bank wages and LFS employment by occupation.

**Why cleaning is needed:**
- The raw CSV has many metadata columns (VECTOR, COORDINATE, SCALAR_ID, etc.).
- Key variables (immigrant status, country of birth, sex, age) come as categorical text.
- Labour-force measures (employment / unemployment / participation) are all in one column.
- We want a consistent window (2015–2024) and a tidy row structure.


In [ ]:
## Step 1 — Initial examination of the raw CSV
We load the file, list columns, and peek at a few rows to understand structure and confirm the presence of:
- `REF_DATE` (period), `GEO` (geography)
- `Immigrant status`, `Country of birth`, `Sex`, `Age group`
- `Labour force characteristics` (the metric)
- `VALUE` (the numeric rate)


In [6]:
import re
from pathlib import Path
import pandas as pd

raw_path = "14100088.csv"   # adjust if needed
df_raw = pd.read_csv(raw_path, low_memory=False)

print("Columns:", df_raw.columns.tolist())
display(df_raw.head(8))
display(df_raw.sample(8, random_state=1))


Columns: ['REF_DATE', 'GEO', 'DGUID', 'Immigrant status', 'Country of birth', 'Labour force characteristics', 'Sex', 'Age group', 'UOM', 'UOM_ID', 'SCALAR_FACTOR', 'SCALAR_ID', 'VECTOR', 'COORDINATE', 'VALUE', 'STATUS', 'SYMBOL', 'TERMINATED', 'DECIMALS']


,REF_DATE,GEO,DGUID,Immigrant status,Country of birth,Labour force characteristics,Sex,Age group,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
0,2006-03,Canada,2016A000011124,Total population,Total population,Population,Both sexes,15 years and over,Persons,249,thousands,3,v53040776,1.1.1.1.1.1,25979.7,NaN,NaN,NaN,1
1,2006-03,Canada,2016A000011124,Total population,Total population,Population,Both sexes,25 to 54 years,Persons,249,thousands,3,v53040777,1.1.1.1.1.2,14084.6,NaN,NaN,NaN,1
2,2006-03,Canada,2016A000011124,Total population,Total population,Population,Males,15 years and over,Persons,249,thousands,3,v53040778,1.1.1.1.2.1,12784.5,NaN,NaN,NaN,1
3,2006-03,Canada,2016A000011124,Total population,Total population,Population,Males,25 to 54 years,Persons,249,thousands,3,v53040779,1.1.1.1.2.2,7035.0,NaN,NaN,NaN,1
4,2006-03,Canada,2016A000011124,Total population,Total population,Population,Females,15 years and over,Persons,249,thousands,3,v53040780,1.1.1.1.3.1,13195.3,NaN,NaN,NaN,1
5,2006-03,Canada,2016A000011124,Total population,Total population,Population,Females,25 to 54 years,Persons,249,thousands,3,v53040781,1.1.1.1.3.2,7049.6,NaN,NaN,NaN,1
6,2006-03,Canada,2016A000011124,Total population,Total population,Labour force,Both sexes,15 years and over,Persons,249,thousands,3,v53040782,1.1.1.2.1.1,17203.9,NaN,NaN,NaN,1
7,2006-03,Canada,2016A000011124,Total population,Total population,Labour force,Both sexes,25 to 54 years,Persons,249,thousands,3,v53040783,1.1.1.2.1.2,12078.2,NaN,NaN,NaN,1


,REF_DATE,GEO,DGUID,Immigrant status,Country of birth,Labour force characteristics,Sex,Age group,UOM,UOM_ID,SCALAR_FACTOR,SCALAR_ID,VECTOR,COORDINATE,VALUE,STATUS,SYMBOL,TERMINATED,DECIMALS
316328,2022-10,Canada,2016A000011124,"Immigrants, landed more than 5 to 10 years ear...",Africa,Labour force,Males,15 years and over,Persons,249,thousands,3,v53042176,1.4.6.2.2.1,71.6,NaN,NaN,NaN,1
61326,2009-05,Canada,2016A000011124,"Immigrants, landed more than 5 to 10 years ear...",Africa,Unemployment rate,Both sexes,15 years and over,Percentage,242,units,0,v53042198,1.4.6.6.1.1,16.5,NaN,NaN,NaN,1
323767,2023-03,Canada,2016A000011124,"Immigrants, landed 5 or less years earlier",Total population,Labour force,Both sexes,25 to 54 years,Persons,249,thousands,3,v53041551,1.3.1.2.1.2,736.3,NaN,NaN,NaN,1
196433,2016-07,Canada,2016A000011124,Total population,Total population,Employment,Females,25 to 54 years,Persons,249,thousands,3,v53040793,1.1.1.3.3.2,5669.5,NaN,NaN,NaN,1
216657,2017-07,Canada,2016A000011124,"Immigrants, landed more than 10 years earlier",Total population,Unemployment rate,Males,25 to 54 years,Percentage,242,units,0,v53042345,1.5.1.6.2.2,5.4,NaN,NaN,NaN,1
143721,2013-09,Canada,2016A000011124,"Immigrants, landed more than 5 to 10 years ear...",Asia,Labour force,Males,25 to 54 years,Persons,249,thousands,3,v53042225,1.4.7.2.2.2,169.0,NaN,NaN,NaN,1
336184,2023-11,Canada,2016A000011124,Landed immigrants,Total population,Participation rate,Females,15 years and over,Percentage,242,units,0,v53041200,1.2.1.7.3.1,60.3,NaN,NaN,NaN,1
180915,2015-09,Canada,2016A000011124,Landed immigrants,Total population,Population,Males,25 to 54 years,Persons,249,thousands,3,v53041163,1.2.1.1.2.2,1763.6,NaN,NaN,NaN,1


In [ ]:
Step 2 — Decide what to keep
We only need analytical variables:
- Time (`REF_DATE`) → we'll convert to `year`
- Geography (`GEO`) → we'll normalize to province codes + "National"
- Demographic dimensions: `Immigrant status`, `Country of birth`, `Sex`, `Age group`
- Measure label: `Labour force characteristics` (Employment rate, Participation rate, Unemployment rate)
- Measure value: `VALUE` (percent)

We drop metadata fields (`VECTOR`, `COORDINATE`, `SCALAR_*`, etc.) because they do not affect analysis.


In [ ]:
## Step 3 — Clean and aggregate to annual (2015–2024)
- Convert `REF_DATE` → `year` and keep 2015–2024
- Map `GEO` to province/territory codes
- Keep only the three headline labour-force metrics
- Convert `VALUE` to numeric
- Average across months within each year (if the table is monthly/3-month moving average)
- Output a tidy table


In [7]:
# Path("data/clean").mkdir(parents=True, exist_ok=True)

def pick_col(df, names_like):
    norm = {re.sub(r"[^a-z0-9]", "", c.lower()): c for c in df.columns}
    for name in names_like:
        key = re.sub(r"[^a-z0-9]", "", name.lower())
        if key in norm:
            return norm[key]
    for c in df.columns:
        if any(n.lower() in c.lower() for n in names_like):
            return c
    return None

df = df_raw.copy()

# Identify columns robustly (handles small naming variations)
date_col  = pick_col(df, ["REF_DATE", "Reference period", "Date"])
geo_col   = pick_col(df, ["GEO", "Geography"])
imm_col   = pick_col(df, ["Immigrant status"])
cob_col   = pick_col(df, ["Country of birth"])
sex_col   = pick_col(df, ["Sex", "Gender"])
age_col   = pick_col(df, ["Age group"])
char_col  = pick_col(df, ["Labour force characteristics"])
val_col   = pick_col(df, ["VALUE", "Value"])

print("Using:", dict(date=date_col, geo=geo_col, imm=imm_col, cob=cob_col,
                    sex=sex_col, age=age_col, char=char_col, value=val_col))

# Keep only relevant columns
keep = [date_col, geo_col, char_col, val_col]
for c in [imm_col, cob_col, sex_col, age_col]:
    if c: keep.append(c)
df = df[keep].copy()

# Convert to year window 2015–2024
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df["year"] = df[date_col].dt.year
df = df[df["year"].between(2015, 2024, inclusive="both")]

# Normalize geography to codes
province_map = {
    "Canada": "National",
    "Newfoundland and Labrador": "NL", "Prince Edward Island": "PE",
    "Nova Scotia": "NS", "New Brunswick": "NB", "Quebec": "QC",
    "Ontario": "ON", "Manitoba": "MB", "Saskatchewan": "SK",
    "Alberta": "AB", "British Columbia": "BC",
    "Yukon": "YT", "Northwest Territories": "NT", "Nunavut": "NU",
}
df["province"] = df[geo_col].replace(province_map)

# Rename key columns
rename = {
    char_col: "lf_characteristic",
    val_col: "value",
    imm_col: "immigrant_status" if imm_col else None,
    cob_col: "country_of_birth" if cob_col else None,
    sex_col: "sex" if sex_col else None,
    age_col: "age_group" if age_col else None,
}
rename = {k:v for k,v in rename.items() if v is not None}
df = df.rename(columns=rename)

# Keep only headline labour-force metrics (case-insensitive match)
wanted = {"employment rate", "unemployment rate", "participation rate",
          "labour force participation rate"}
df = df[df["lf_characteristic"].str.lower().isin(wanted)]

# Ensure numeric
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Annualize: mean of months in each year across selected dimensions
group_cols = ["year", "province", "lf_characteristic"]
for opt in ["immigrant_status", "country_of_birth", "sex", "age_group"]:
    if opt in df.columns:
        group_cols.append(opt)

annual = (df.groupby(group_cols, dropna=False)["value"]
            .mean()
            .reset_index())

# Order columns nicely
ordered = [c for c in ["year","province","lf_characteristic",
                       "immigrant_status","country_of_birth","sex","age_group","value"]
           if c in annual.columns]
annual = annual[ordered]

print("Rows:", len(annual))
print("Years:", sorted(annual["year"].unique()))
print("Characteristics:", annual["lf_characteristic"].unique().tolist())
display(annual.head(10))


Using: {'date': 'REF_DATE', 'geo': 'GEO', 'imm': 'Immigrant status', 'cob': 'Country of birth', 'sex': 'Sex', 'age': 'Age group', 'char': 'Labour force characteristics', 'value': 'VALUE'}
Rows: 5940
Years: [np.int32(2015), np.int32(2016), np.int32(2017), np.int32(2018), np.int32(2019), np.int32(2020), np.int32(2021), np.int32(2022), np.int32(2023), np.int32(2024)]
Characteristics: ['Employment rate', 'Participation rate', 'Unemployment rate']


,year,province,lf_characteristic,immigrant_status,country_of_birth,sex,age_group,value
0,2015,National,Employment rate,Born in Canada,Canada,Both sexes,15 years and over,62.808333
1,2015,National,Employment rate,Born in Canada,Canada,Both sexes,25 to 54 years,83.291667
2,2015,National,Employment rate,Born in Canada,Canada,Females,15 years and over,59.625000
3,2015,National,Employment rate,Born in Canada,Canada,Females,25 to 54 years,81.083333
4,2015,National,Employment rate,Born in Canada,Canada,Males,15 years and over,65.991667
5,2015,National,Employment rate,Born in Canada,Canada,Males,25 to 54 years,85.466667
6,2015,National,Employment rate,Born in Canada,Total population,Both sexes,15 years and over,62.808333
7,2015,National,Employment rate,Born in Canada,Total population,Both sexes,25 to 54 years,83.291667
8,2015,National,Employment rate,Born in Canada,Total population,Females,15 years and over,59.625000
9,2015,National,Employment rate,Born in Canada,Total population,Females,25 to 54 years,81.083333


In [8]:
# Step 4 — Save the cleaned dataset
# We write a tidy CSV that can be joined (by year/province) to other sources 
# (Job Bank wages, LFS employment by occupation) or used standalone for equity analysis.

out_path = "LFS_demographics_2015_2024_clean.csv"
annual.to_csv(out_path, index=False)
print("[SAVED]", out_path)



[SAVED] LFS_demographics_2015_2024_clean.csv


In [9]:
# Step 5 — Sanity checks
# Quick summaries to verify the data look reasonable.

# Missingness by column
print(annual.isna().mean().sort_values(ascending=False).round(3))

# Example: immigrant vs born in Canada, national employment rate trend
check = (annual[(annual["province"]=="National") &
                 (annual["lf_characteristic"].str.contains("Employment rate", case=False)) &
                 (annual["immigrant_status"].notna())]
         .pivot_table(index="year", columns="immigrant_status", values="value"))
display(check.head(10))


value                0.014
year                 0.000
province             0.000
lf_characteristic    0.000
immigrant_status     0.000
country_of_birth     0.000
sex                  0.000
age_group            0.000
dtype: float64


immigrant_status,Born in Canada,"Immigrants, landed 5 or less years earlier","Immigrants, landed more than 10 years earlier","Immigrants, landed more than 5 to 10 years earlier",Landed immigrants,Total population
year,,,,,,
2015,39.565278,41.809419,37.719665,43.454843,37.954580,38.219940
2016,39.479167,41.091584,38.374050,43.013191,38.031404,38.356205
2017,39.484028,41.911675,38.299140,43.206551,38.379410,38.539601
2018,39.362500,41.634315,37.834236,44.743813,38.181267,38.268525
2019,39.438194,42.968136,38.628443,44.156675,38.472608,38.615774
2020,39.204861,41.845770,37.987269,41.668659,38.262222,38.350794
2021,39.305556,44.397891,38.267986,43.277888,38.890201,39.053274
2022,39.244444,43.580177,38.440471,44.813848,39.109645,39.069246
2023,39.231944,44.090642,38.566090,44.416519,39.200270,39.117044


In [ ]:
## 6 Why this approach
- **Reproducibility:** Every transformation (filtering, renaming, annualization) is scripted.
- **Alignment:** We constrain years to 2015–2024 to align with Job Bank wages and LFS employment.
- **Tidy structure:** One row per (year × province × group × labour-force metric) enables clean plotting, pivoting, and joining.
- **Relevance:** We keep only headline labour metrics that speak directly to “who benefits/left behind” (employment, unemployment, participation).


In [ ]:
## 7 Optional — Quick plot: Employment rate (National),import matplotlib.pyplot as plt

subset = (annual[(annual["province"]=="National") &
                 (annual["lf_characteristic"].str.contains("Employment rate", case=False)) &
                 (annual["immigrant_status"].notna())])

pivot = subset.pivot_table(index="year", columns="immigrant_status", values="value")
pivot.plot(figsize=(9,5))
plt.ylabel("Employment rate (%)")
plt.xlabel("Year")
plt.title("Employment rate: Immigrants vs Born in Canada (National)")
plt.legend(title="")
plt.show()
 Immigrants vs Born in Canada


In [ ]:
Awesome — here’s a clear, copy-paste notebook you can run to **show your whole workflow** from the raw StatCan CSV to the cleaned, analysis-ready demography dataset.

> Replace paths if yours differ. Each step has a short Markdown cell (explanation) followed by a Code cell (what you ran).

---

# 🧭 0) Title & purpose (Markdown)

```
# LFS Demography Cleaning (2015–2024)
**Goal:** Turn StatCan table `14100088.csv` (Labour force characteristics by immigrant status / country of birth) into a tidy, annual dataset suitable for analysis alongside Job Bank wages and LFS employment by occupation.

**Why cleaning is needed:**
- The raw CSV has many metadata columns (VECTOR, COORDINATE, SCALAR_ID, etc.).
- Key variables (immigrant status, country of birth, sex, age) come as categorical text.
- Labour-force measures (employment / unemployment / participation) are all in one column.
- We want a consistent window (2015–2024) and a tidy row structure.
```

---

# 🔍 1) Load & inspect raw file (Markdown)

```
## Step 1 — Initial examination of the raw CSV
We load the file, list columns, and peek at a few rows to understand structure and confirm the presence of:
- `REF_DATE` (period), `GEO` (geography)
- `Immigrant status`, `Country of birth`, `Sex`, `Age group`
- `Labour force characteristics` (the metric)
- `VALUE` (the numeric rate)
```

**Code**

```python
import pandas as pd

raw_path = "data/raw/14100088.csv"   # adjust if needed
df_raw = pd.read_csv(raw_path, low_memory=False)

print("Columns:", df_raw.columns.tolist())
display(df_raw.head(8))
display(df_raw.sample(8, random_state=1))
```

---

# 🧠 2) What we keep & why (Markdown)

```
## Step 2 — Decide what to keep
We only need analytical variables:
- Time (`REF_DATE`) → we'll convert to `year`
- Geography (`GEO`) → we'll normalize to province codes + "National"
- Demographic dimensions: `Immigrant status`, `Country of birth`, `Sex`, `Age group`
- Measure label: `Labour force characteristics` (Employment rate, Participation rate, Unemployment rate)
- Measure value: `VALUE` (percent)

We drop metadata fields (`VECTOR`, `COORDINATE`, `SCALAR_*`, etc.) because they do not affect analysis.
```

---

# 🧼 3) Cleaning & annualization (Markdown)

```
## Step 3 — Clean and aggregate to annual (2015–2024)
- Convert `REF_DATE` → `year` and keep 2015–2024
- Map `GEO` to province/territory codes
- Keep only the three headline labour-force metrics
- Convert `VALUE` to numeric
- Average across months within each year (if the table is monthly/3-month moving average)
- Output a tidy table
```

**Code**

```python
import re
from pathlib import Path

Path("data/clean").mkdir(parents=True, exist_ok=True)

def pick_col(df, names_like):
    norm = {re.sub(r"[^a-z0-9]", "", c.lower()): c for c in df.columns}
    for name in names_like:
        key = re.sub(r"[^a-z0-9]", "", name.lower())
        if key in norm:
            return norm[key]
    for c in df.columns:
        if any(n.lower() in c.lower() for n in names_like):
            return c
    return None

df = df_raw.copy()

# Identify columns robustly (handles small naming variations)
date_col  = pick_col(df, ["REF_DATE", "Reference period", "Date"])
geo_col   = pick_col(df, ["GEO", "Geography"])
imm_col   = pick_col(df, ["Immigrant status"])
cob_col   = pick_col(df, ["Country of birth"])
sex_col   = pick_col(df, ["Sex", "Gender"])
age_col   = pick_col(df, ["Age group"])
char_col  = pick_col(df, ["Labour force characteristics"])
val_col   = pick_col(df, ["VALUE", "Value"])

print("Using:", dict(date=date_col, geo=geo_col, imm=imm_col, cob=cob_col,
                    sex=sex_col, age=age_col, char=char_col, value=val_col))

# Keep only relevant columns
keep = [date_col, geo_col, char_col, val_col]
for c in [imm_col, cob_col, sex_col, age_col]:
    if c: keep.append(c)
df = df[keep].copy()

# Convert to year window 2015–2024
df[date_col] = pd.to_datetime(df[date_col], errors="coerce")
df["year"] = df[date_col].dt.year
df = df[df["year"].between(2015, 2024, inclusive="both")]

# Normalize geography to codes
province_map = {
    "Canada": "National",
    "Newfoundland and Labrador": "NL", "Prince Edward Island": "PE",
    "Nova Scotia": "NS", "New Brunswick": "NB", "Quebec": "QC",
    "Ontario": "ON", "Manitoba": "MB", "Saskatchewan": "SK",
    "Alberta": "AB", "British Columbia": "BC",
    "Yukon": "YT", "Northwest Territories": "NT", "Nunavut": "NU",
}
df["province"] = df[geo_col].replace(province_map)

# Rename key columns
rename = {
    char_col: "lf_characteristic",
    val_col: "value",
    imm_col: "immigrant_status" if imm_col else None,
    cob_col: "country_of_birth" if cob_col else None,
    sex_col: "sex" if sex_col else None,
    age_col: "age_group" if age_col else None,
}
rename = {k:v for k,v in rename.items() if v is not None}
df = df.rename(columns=rename)

# Keep only headline labour-force metrics (case-insensitive match)
wanted = {"employment rate", "unemployment rate", "participation rate",
          "labour force participation rate"}
df = df[df["lf_characteristic"].str.lower().isin(wanted)]

# Ensure numeric
df["value"] = pd.to_numeric(df["value"], errors="coerce")

# Annualize: mean of months in each year across selected dimensions
group_cols = ["year", "province", "lf_characteristic"]
for opt in ["immigrant_status", "country_of_birth", "sex", "age_group"]:
    if opt in df.columns:
        group_cols.append(opt)

annual = (df.groupby(group_cols, dropna=False)["value"]
            .mean()
            .reset_index())

# Order columns nicely
ordered = [c for c in ["year","province","lf_characteristic",
                       "immigrant_status","country_of_birth","sex","age_group","value"]
           if c in annual.columns]
annual = annual[ordered]

print("Rows:", len(annual))
print("Years:", sorted(annual["year"].unique()))
print("Characteristics:", annual["lf_characteristic"].unique().tolist())
display(annual.head(10))
```

---

# ✅ 4) Save and document the output (Markdown)

```
## Step 4 — Save the cleaned dataset
We write a tidy CSV that can be joined (by year/province) to other sources (Job Bank wages, LFS employment by occupation) or used standalone for equity analysis.
```

**Code**

```python
out_path = "data/clean/LFS_demographics_2015_2024_clean.csv"
annual.to_csv(out_path, index=False)
print("[SAVED]", out_path)
```

---

# 🧪 5) Sanity checks (Markdown)

```
## Step 5 — Sanity checks
Quick summaries to verify the data look reasonable.
```

**Code**

```python
# Missingness by column
print(annual.isna().mean().sort_values(ascending=False).round(3))

# Example: immigrant vs born in Canada, national employment rate trend
check = (annual[(annual["province"]=="National") &
                 (annual["lf_characteristic"].str.contains("Employment rate", case=False)) &
                 (annual["immigrant_status"].notna())]
         .pivot_table(index="year", columns="immigrant_status", values="value"))
display(check.head(10))
```

---

# 🧩 6) Rationale (Markdown)

```
## Why this approach
- **Reproducibility:** Every transformation (filtering, renaming, annualization) is scripted.
- **Alignment:** We constrain years to 2015–2024 to align with Job Bank wages and LFS employment.
- **Tidy structure:** One row per (year × province × group × labour-force metric) enables clean plotting, pivoting, and joining.
- **Relevance:** We keep only headline labour metrics that speak directly to “who benefits/left behind” (employment, unemployment, participation).
```

---

# 📈 7) (Optional) One quick plot (Markdown)

```
## Optional — Quick plot: Employment rate (National), Immigrants vs Born in Canada
```

**Code**

```python
import matplotlib.pyplot as plt

subset = (annual[(annual["province"]=="National") &
                 (annual["lf_characteristic"].str.contains("Employment rate", case=False)) &
                 (annual["immigrant_status"].notna())])

pivot = subset.pivot_table(index="year", columns="immigrant_status", values="value")
pivot.plot(figsize=(9,5))
plt.ylabel("Employment rate (%)")
plt.xlabel("Year")
plt.title("Employment rate: Immigrants vs Born in Canada (National)")
plt.legend(title="")
plt.show()
```

---

## What to say in your report (you can copy)

* *“We began with StatCan table 14100088 (CSV) and inspected the schema to identify analytical variables (REF\_DATE, GEO, Immigrant status, Country of birth, Sex, Age group, Labour force characteristics, VALUE). We removed metadata fields and standardized geography to province codes. We limited the period to 2015–2024 to align with other sources. We converted REF\_DATE to year and aggregated monthly/3-month averages to annual means for comparability. We retained three headline metrics (employment, participation, unemployment rates) as they capture access to work and barriers for different demographic groups. The result is a tidy dataset suitable for equity analysis alongside occupation wages and employment.”*

If anything doesn’t match your exact column names, tell me the column list it prints and I’ll tweak the `pick_col` step.
